In [1]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [2]:
import galois
import numpy as np
import time

In [3]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(2**3)

In [4]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [5]:
#ho verificato che funziona pure con i random
#A = field.Random((k, n_k))
#print(A)

#questa è con l'esempio di 4.3
A = field([[1,2,4,7],
           [0,2,3,1],
           [2,3,2,0]])

In [6]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [7]:
#np.dot prodotto di due  array
A_primo_intermedio = field(np.dot(matrice_bin1, A))
print(A_primo_intermedio)

[[2 3 2 0]
 [1 2 4 7]
 [0 2 3 1]]


In [8]:
A_primo = field(np.dot(A_primo_intermedio, matrice_bin2))
print(A_primo)

[[2 0 3 2]
 [1 7 2 4]
 [0 1 2 3]]


In [9]:
import threading

In [10]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 
#POTREI PER ESEMPIO PENSARE DI SALVARMI I PRIMI Y CHE SONO MEDIAMENTE I CONFLITTI CHE TROVIAMO E QUINDI UNA VOLTA SUPERATA
#LA Y-ESIMA UGUAGLIANZA TRA DUE COLONNE DIVERSE ANDIAMO AD ESEGUIRE A RUN TIME ALTRI CONFRONTI

#creo il lock per evitare conglitti nell'aggiornamento
lock = threading.Lock()

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    somma = field(np.sum([riga])%8)

    #print("Riga : ", riga) #questi vanno eliminati poi
    #print("Tipo riga: ", type(riga)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print("Somma: ", somma) #questi vanno eliminati poi
    #print("Tipo somma: ",type(somma)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    with lock: #vado ad usare il lock
        somme_rig[indice_riga] = somma
        #print(somme_rig)

#vado ad estrarre il primo ed il secondo elemento di ogni colonna
def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    with lock:
        primi_colonne[indice_colonna] = colonna[0]
        #print(primi_colonne)
        secondi_colonne[indice_colonna] = colonna[1]
        #print(secondi_colonne)

#eseguo in parallelo somma_riga
def esegui_somma_righe_parallelo(matrice):
    threads_righe = []

    for indice_riga in range(matrice.shape[0]):
        t = threading.Thread(target = somma_riga, args = (matrice, indice_riga))
        threads_righe.append(t)
        t.start() #avvio il thread

    #aspetto il risultato di tutti i threads
    for t in threads_righe:
        t.join()

#eseguo in parallelo estrazione
def esegui_estrazione_parallelo(matrice):
    threads_colonne = []

    for indice_colonna in range(matrice.shape[1]):
        t = threading.Thread(target = estrai_da_colonna, args = (matrice, indice_colonna))
        threads_colonne.append(t)
        t.start() #avvio il thread

    #aspetto il risultato di tutti i threads
    for t in threads_colonne:
        t.join()


In [11]:
#threads_righe = []
#threds_colonne = []
colonne_finali = []

In [12]:
esegui_somma_righe_parallelo(A_primo)

In [13]:
print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)
print ("\nElementi primi estratti: ", primi_colonne)
print ("\nElementi secondi estratti: ", secondi_colonne)


Somma degli elementi di ciascuna riga:  [GF(7, order=2^3), GF(6, order=2^3), GF(6, order=2^3)]

Elementi primi estratti:  [0, 0, 0, 0]

Elementi secondi estratti:  [0, 0, 0, 0]


In [14]:
#ordino in maniera crescente per valore gli indici
indici_ordinati = np.argsort(somme_rig)
print(indici_ordinati)

#applico gli swap
A_canonica_righe = A_primo[indici_ordinati, :]

print(A_canonica_righe)

[1 2 0]
[[1 7 2 4]
 [0 1 2 3]
 [2 0 3 2]]


In [15]:
#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1]))

#qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
A_CF = A_canonica_righe[:, [col[2] for col in colonne_finali]]

In [16]:
print(A_CF)

[[1 2 4 7]
 [0 2 3 1]
 [2 3 2 0]]


In [17]:
#devo fare lo stesso per la matrice A per dimostrare che sono CF

start_time= time.perf_counter()
colonne_finali = []

#creo un thread per ogni colonna
esegui_somma_righe_parallelo(A)

#print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
#print(indici_ordinati)

#applico gli swap
A_canonica_righe = A[indici_ordinati, :]

#print(A_canonica_righe)

#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1]))

A_CF = A_canonica_righe[:, [col[2] for col in colonne_finali]]

end_timer = time.perf_counter()
durata = (end_timer - start_time)*1000
print("Durata esecuzione: ", durata, " ms")

Durata esecuzione:  157.7864870000667  ms


In [18]:
print(A_CF)

[[1 2 4 7]
 [0 2 3 1]
 [2 3 2 0]]
